In [15]:
import configparser
import nltk
import xml.etree.ElementTree as ET
import unicodedata
from unidecode import unidecode
import string
import re
import csv
from tqdm import tqdm

In [20]:
config = configparser.ConfigParser()
config.read('PC.cfg')

query_input_file = config.get('DEFAULT', 'LEIA')
query_output_file = config.get('DEFAULT', 'CONSULTAS')
esperados_output_file = config.get('DEFAULT', 'ESPERADOS')

def parse_xml(file_path, first_field, second_field):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    xml_dict = {}
    
    for query in root.findall('QUERY'):
        key = query.find(first_field).text.strip()
        value = query.find(second_field).text.strip()
        xml_dict[key] = value
        
    return queries

queries = parse_xml(query_input_file, 'QueryNumber', 'QueryText')
# print(queries)


In [17]:
def format_text(text):
    text = text.upper()
    # Remove accents
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove newlines and extra spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [18]:
formatted_queries = {key: format_text(value) for key, value in queries.items()}
# formatted_queries

In [21]:
def write_queries_to_csv(queries, output_file):
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["QueryNumber", "QueryText"])
        for query_number, query_text in queries.items():
            writer.writerow([query_number, query_text])

write_queries_to_csv(formatted_queries, query_output_file)
            